In [ ]:
from sko.GA import GA
import keras
import numpy as np
import pandas as pd
from sko.tools import set_run_mode
import datetime

In [ ]:
loaded_model=keras.models.load_model('Neutral_OER_transAL.h5')

In [ ]:
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
Minmaxsc  = MinMaxScaler(feature_range=(0, 1))
Minmaxsc2  = MinMaxScaler(feature_range=(0, 1))
Stdsc  = StandardScaler()
Stdsc2  = StandardScaler()
MAsc  = MaxAbsScaler()
MAsc2  = MaxAbsScaler()
Rsc  = RobustScaler()
Rsc2  = RobustScaler()

In [ ]:
database=pd.read_csv('processed_database.csv')
data_output_full=database.iloc[:,1]
data_input_full=database.iloc[:,2:]
data_input_full_ANN=Stdsc.fit_transform(data_input_full)
data_output_full_ANN=Stdsc2.fit_transform(np.array(data_output_full).reshape(-1,1))

In [ ]:
element_information={}
element_information['None']=[0,0,0,0,0,0,0,0]
element_information['O']=[16,8,2,16,13.56,3.44,0,48]

element_information['Sc']=[44.956, 21, 4, 3, 6.57, 1.36, 1, 162]
element_information['Ti']=[47.867, 22, 4, 4, 6.81, 1.54, 2, 147]
element_information['V']=[50.942, 23, 4, 5, 6.76, 1.63, 3, 134]
element_information['Cr']=[51.996, 24, 4, 6, 6.74, 1.66, 4, 130]
element_information['Mn']=[54.938, 25, 4, 7, 7.4, 1.56, 5, 127]
element_information['Fe']=[55.845, 26, 4, 8, 7.83, 1.83, 6, 126]
element_information['Co']=[58.933, 27, 4, 9, 7.81, 1.88, 7, 125]
element_information['Ni']=[58.6934, 28, 4, 10, 7.61, 1.91, 8, 124]
element_information['Cu']=[63.546, 29, 4, 11, 7.69, 1.9, 10, 128]
element_information['Zn']=[65.38, 30, 4, 12, 9.35, 1.65, 10, 138]

element_information['Zr']=[91.224, 40, 5, 4, 6, 1.33, 2, 160]
element_information['Nb']=[92.906, 41, 5, 5, 6.76, 1.6, 4, 146]
element_information['Mo']=[95.94, 42, 5, 6, 7.35, 2.16, 5, 139]
element_information['Tc']=[98.906, 43, 5, 7, 7.28, 1.9, 5, 136]
element_information['Ru']=[101.07, 44, 5, 8, 7.5, 2.2, 7, 134]
element_information['Rh']=[102.905, 45, 5, 9, 7.7, 2.28, 8, 134]
element_information['Pd']=[106.42, 46, 5, 10, 8.3, 2.2, 10, 137]
element_information['Ag']=[107.868, 47, 5, 11, 7.54, 1.93, 10, 144]
element_information['Cd']=[112.411, 48, 5, 12, 8.95, 1.69, 10, 154]

element_information['Ta']=[180.947,73, 6, 5, 7.89, 1.5, 3, 149]
element_information['W']=[183.84, 74, 6, 6, 7.98, 2.36, 4, 141]
element_information['Re']=[186.207, 75, 6, 7, 7.88, 1.9, 5, 137]
element_information['Os']=[190.23, 76, 6, 8, 8.7, 2.2, 6, 135]
element_information['Ir']=[192.217, 77, 6, 9, 9.1, 2.2, 7, 136]
element_information['Pt']=[195.084, 78, 6, 10, 8.9, 2.28, 9, 139]
element_information['Au']=[196.966, 79, 6, 11, 9.19, 2.54, 10, 144]
###added

In [ ]:
element_list=list(element_information.keys())

In [ ]:
len(element_list)

In [ ]:
def predict_func_OVP(p):
    ele_1,ele_2,ele_3,mat1,mat2,mat3,mat4,mat5,mat6,mat7,mat8,mat9,mat10,mat11,mat12,mat13,mat14,mat15,mat16,mat17,mat18,mat19,mat20,mat21,mat22,mat23,mat24,mat25,mat26,mat27,mat28,mat29,mat30,mat31,mat32,mat33,mat34,mat35,mat36,mat37,mat38,mat39,mat40,mat41,mat42,mat43,mat44,mat45,mat46,mat47,mat48,mat49,mat50=p
    #####
    ele_info=element_information[element_list[int(ele_1)]]+element_information[element_list[int(ele_2)]]+element_information[element_list[int(ele_3)]]+element_information[element_list[0]]+element_information[element_list[0]]+element_information[element_list[0]]+element_information[element_list[0]]+element_information[element_list[0]]
    mater_prop_info=[mat1,mat2,mat3,mat4,mat5,mat6,mat7,mat8,mat9,mat10,mat11,mat12,mat13,mat14,mat15,mat16,mat17,mat18,mat19,mat20,mat21,mat22,mat23,mat24,mat25,mat26,mat27,mat28,mat29,mat30,mat31,mat32,mat33,mat34,mat35,mat36,mat37,mat38,mat39,mat40,mat41,mat42,mat43,mat44,mat45,mat46,mat47,mat48,mat49,mat50]
    ###合并制备条件
    info_all=ele_info+mater_prop_info
    input_array=np.array(info_all)
    input_array=input_array.reshape(1, -1)
    input_array=pd.DataFrame(input_array,columns=data_input_full.columns)
    #####预测
    prediction=loaded_model.predict(input_array)[0]
#     print(prediction)
    return prediction[0]

In [ ]:
def uneq_element(x):
    return (x[1]!=x[0] and x[1]!=x[2] and x[0]!=x[2])-1

constraint_eq = [
    uneq_element
    ]

constraint_ueq = [
    lambda x: (x[3]+x[4]+x[5]+x[6]+x[7]+x[8]+x[9]+x[10]+x[11]+x[12]+x[13]+x[14]+x[15]+x[16]+x[17]+x[18]+x[19]+x[20]+x[21]+x[22]+x[23]+x[24]+x[25]+x[26]+x[27]+x[28]+x[29]+x[30]+x[31]+x[32]+x[33]+x[34]+x[35]+x[36]+x[37]+x[38]+x[39]+x[40]+x[41]+x[42]+x[43]+x[44]+x[45]+x[46]+x[47]+x[48]+x[49]+x[50]+x[51]+x[52])-3.9
]


def generate_costly_function(task_type='io_costly'):
    # generate a high cost function to test all the modes
    # cost_type can be 'io_costly' or 'cpu_costly'
    if task_type == 'io_costly':
        def costly_function():
            time.sleep(0.1)
            return 1
    else:
        def costly_function():
            n = 10000
            step1 = [np.log(i + 1) for i in range(n)]
            step2 = [np.power(i, 1.1) for i in range(n)]
            step3 = sum(step1) + sum(step2)
            return step3

    return costly_function
from numpy import *

In [ ]:
def explain_best_x(best_x):
    print('explaination:')
    for each_ele_serial in list(best_x)[0:3]:
        print(element_list[int(each_ele_serial)])
    for i in range(3,53):
        if list(best_x)[i]==1:
            print(mat[i-3])

In [ ]:
mat=list(data_input_full.columns)[64:]

In [ ]:
for i in range(1,7):
    task_type='cpu_costly'
    costly_function = generate_costly_function(task_type=task_type)
    # to use the vectorization mode, the function itself should support the mode.
    mode = 'common'
    # mode ='vectorization'
    set_run_mode(predict_func_OVP, mode)
    ga = GA(func=predict_func_OVP, n_dim=53, size_pop=200, max_iter=20,prob_mut=0.01,
            lb=[1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
            ub=[25,25,25,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            constraint_eq=constraint_eq,constraint_ueq=constraint_ueq, precision=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
    start_time = datetime.datetime.now()
    best_x, best_y = ga.run()
    print('best_x:', best_x, '\n', 'best_y:', best_y)
    print('on {task_type} task,use {mode} mode, costs {time_costs}s'
          .format(task_type=task_type, mode=mode,
                  time_costs=(datetime.datetime.now() - start_time).total_seconds()))
    explain_best_x(best_x)